# Foodie Recommender Data Model (V0.1)
We'll choose a set of starter features and attempt to train a two-towered recommender system

In [15]:
import dateutil
from pydantic import BaseModel, validator
from typing import Any, List

import json


genres = {}



def save_model_data(model: BaseModel, filename: str):
    with open(filename, 'w+') as file:
        json.dump(model.json(), file)

        

def add_element_to_genres(element: str, genres: dict = genres) -> int:
    dict_len = len(genres)
    if element not in genres:
        genres[element] = dict_len + 1
    return genres[element]
        
        
        
def map_genre(genre_list: list) -> list:
    tmp = []
    for element in genre_list:
        tmp.append(add_element_to_genres(element))
    return tmp
        

    
class RestaurantUser(BaseModel):
    user_birth_date: int
    user_genres: List[Any] = [0]
    user_id: int
    user_occupation: str
    user_gender: bool  # 0: male, 1: female
    user_zip_code: int
    
    
    @validator('user_genres')
    def index_or_add(cls, v):
        assert len(v) > 0, 'Must provide list of genre > 0'
        return map_genre(v)
    
    def save(self, name='users.json') -> None:
        return save_model_data(self, name)

    
        
class Restaurant(BaseModel):
    restaurant_id: int
    restaurant_title: str
    restaurant_genres: List[Any]
    restaurant_zip_code: int
   
    @validator('restaurant_genres')
    def index_or_add(cls, v):
        assert len(v) > 0, 'Must provide list of genre > 0'
        return map_genre(v)
        
    def save(self, name='users.json') -> None:
        return save_model_data(self, name)
    
    
    
class RestaurantRating(BaseModel):
    user: RestaurantUser
    restaurant: Restaurant
    timestamp: int  # Converting all date/time to posix integer
    restaurant_rating: int
    
    @validator('timestamp')
    def convert_valid_time(cls, v: str):
        pass
    
    def save(self, name='users.json') -> None:
        return save_model_data(self, name)

    
    

In [18]:
# Create a test user
user_profile = {
    "user_birth_date": 20220101,
    "user_genres": ['vegetarian', 'thai'],
    "user_id": 1001,
    "user_occupation": "student",
    "user_gender": 0,  # 0: male, 1: female
    "user_zip_code": 84104,
}

user = RestaurantUser(**user_profile)

# Create a restaurant
restaurant_profile

In [19]:
user

RestaurantUser(user_birth_date=20220101, user_genres=[1, 2], user_id=1001, user_occupation='student', user_gender=False, user_zip_code=84104)